In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/My Drive/589MiniProject/")
%cd /content/drive/My Drive/589MiniProject/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1yyh1nEBAISidzfrj3Aql57NwD7xFpa20/589MiniProject


In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchsummary import summary
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import os
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 75

print('using device:', device)

using device: cpu


In [4]:
!pip install img2vec_pytorch

  Created wheel for img2vec-pytorch: filename=img2vec_pytorch-0.2.5-py3-none-any.whl size=4345 sha256=d8b38b2c6e389c2d883d67dabe45f7a9b290a6a816a34fb31dd70a8846a09a62
  Stored in directory: /root/.cache/pip/wheels/07/4d/bf/9de03c3e76c20899b69498f90624ec499c7c67dd13f183f4e9
Successfully built img2vec-pytorch


In [5]:
from img2vec_pytorch import Img2Vec
from PIL import Image

# # Initialize Img2Vec with GPU
# img2vec = Img2Vec(torch.cuda.is_available())

# # Read in an image (rgb format)
# img = Image.open('test.jpg')
# # Get a vector from img2vec, returned as a torch FloatTensor
# vec = img2vec.get_vec(img, tensor=True)
# # Or submit a list
# vectors = img2vec.get_vec(list_of_PIL_images)
def imgVec(folder='Train'):
  dataPath = "Data/" + folder + "/"
  metadataPath = "./Data/MetaData/" + folder.lower() + ".csv"
  df = pd.read_csv(metadataPath)
  # df['img_path'] = os.getcwd() + '/Data/' + folder + "/" + df['Id'] + '.jpg'
  df['img_path'] = './Data/' + folder + "/" + df['Id'] + '.jpg'
  img2vec = Img2Vec(cuda=(torch.cuda.is_available()))  
  img_vecs = []
  for idx, row in df.iterrows():
    # print(path + '/Data/' + 'Test/' + row['Id'] + '.jpg')
    img = Image.open(row["img_path"])
    vec = img2vec.get_vec(img)
    img_vecs.append(vec)

  df['feature_vector'] = img_vecs
  return df


In [6]:
train_df = imgVec('Train')
train_df.reset_index(inplace=True)
col_names = ["feat"+str(i) for i in range(len(train_df.iloc[0]["feature_vector"]))]
train_df[col_names] = pd.DataFrame(train_df.feature_vector.tolist(), index= train_df.index)
train_df.drop(columns = ["Id","img_path","feature_vector"], inplace=True)


valid_df = imgVec('Valid')
valid_df.reset_index(inplace=True)
col_names = ["feat"+str(i) for i in range(len(valid_df.iloc[0]["feature_vector"]))]
valid_df[col_names] = pd.DataFrame(valid_df.feature_vector.tolist(), index= valid_df.index)
valid_df.drop(columns = ["Id","img_path","feature_vector"], inplace=True)

test_df = imgVec('Test')
test_df.reset_index(inplace=True)
col_names = ["feat"+str(i) for i in range(len(test_df.iloc[0]["feature_vector"]))]
test_df[col_names] = pd.DataFrame(test_df.feature_vector.tolist(), index= test_df.index)
test_df.drop(columns = ["Id","img_path","feature_vector"], inplace=True)

print("Train ...")
print(train_df.head())

print("Valid ...")
print(valid_df.head())

print("Test ...")
print(test_df.head())

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Train ...
   index  Subject Focus  Eyes  Face  ...   feat508   feat509   feat510   feat511
0      0              0     1     1  ...  1.552251  0.729470  1.408372  1.468240
1      1              0     1     1  ...  0.841265  1.340379  2.015083  0.379474
2      2              0     1     1  ...  0.902725  0.322946  1.937619  0.461102
3      3              0     1     1  ...  1.789118  1.036564  0.068066  0.716284
4      4              0     1     1  ...  3.102909  1.097454  0.581716  0.273106

[5 rows x 526 columns]
Valid ...
   index  Subject Focus  Eyes  Face  ...   feat508   feat509   feat510   feat511
0      0              0     0     1  ...  1.847430  0.734269  0.584828  1.115096
1      1              0     1     1  ...  1.499832  2.500482  0.277388  0.397272
2      2              0     1     1  ...  0.401953  0.236270  0.914187  0.319195
3      3              0     1     1  ...  2.246782  1.693573  0.013458  1.673211
4      4              0     1     1  ...  2.828182  1.301023  0.4

In [9]:
train_y, valid_y, test_y = train_df['Pawpularity'], valid_df['Pawpularity'], test_df['Pawpularity']
train_X = train_df.drop(columns=['Pawpularity'])
train_X = preprocessing.StandardScaler().fit_transform(train_X)
valid_X = valid_df.drop(columns=['Pawpularity'])
valid_X = preprocessing.StandardScaler().fit_transform(valid_X)
test_X = test_df.drop(columns=['Pawpularity'])
test_X = preprocessing.StandardScaler().fit_transform(test_X)

In [10]:
# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    # return torch.from_numpy(df.values).type(dtype).to(device)
    # return torch.from_numpy(df.to_numpy()).type(dtype).to(device)
    return torch.from_numpy(df).type(dtype).to(device)

In [11]:
def df_to_tensor_labels(df):
    return torch.from_numpy(df.to_numpy()).type(dtype).to(device)

In [12]:
torch_train_X = df_to_tensor(train_X)
torch_train_y = df_to_tensor_labels(train_y)
train_ds = TensorDataset(torch_train_X, torch_train_y)

torch_valid_X = df_to_tensor(valid_X)
torch_valid_y = df_to_tensor_labels(valid_y)
valid_ds = TensorDataset(torch_valid_X, torch_valid_y)

torch_test_X = df_to_tensor(test_X)
torch_test_y = df_to_tensor_labels(test_y)
test_ds = TensorDataset(torch_test_X, torch_test_y)

In [13]:
train_loader = DataLoader(train_ds, batch_size=64,shuffle=True,num_workers=2)
valid_loader = DataLoader(valid_ds, batch_size=64,shuffle=True,num_workers=2)
test_loader = DataLoader(test_ds, batch_size=64,shuffle=True,num_workers=2)

In [15]:
loss_history = []

In [16]:
def check_mse(loader,model):
  print('calculating mse for valid set')
  model.eval()
  valid_mse = 0.0
  num_samples = 0
  with torch.no_grad():
      for data, target in loader:
          data, target = data.to(device=device,dtype=dtype), target.to(device=device,dtype=dtype)
          output = model(data)
          num_samples += len(output)
          valid_mse += mean_squared_error(target,output)

  return valid_mse

In [17]:
def train(model, optimizer, epoch=1):
    model.train()
    best_mse = float('inf')
    for e in range(epoch):
      for batch_idx, (data, target) in enumerate(train_loader):
          data, target = data.to(device=device,dtype=dtype), target.to(device=device,dtype=dtype)
          optimizer.zero_grad()
          output = model(data)
          loss = F.mse_loss(output, target)
          loss.backward()
          optimizer.step()

          torch.cuda.empty_cache()
          if batch_idx % 100 == 0:
              # Print loss every 100 batch
              print('Train Epoch: {}\tLoss: {:.6f}'.format(
                  e, loss.item()))
              valid_mse = check_mse(valid_loader,model)
              print('valid mse:',valid_mse)
              if valid_mse < best_mse:
                best_mse = valid_mse
                torch.save(model.state_dict(), "./Models/FCNN.pt")
              torch.cuda.empty_cache()

In [34]:
input_dim = train_X.shape[1]
model = nn.Sequential(
    nn.Linear(input_dim,150,bias=True),
    nn.BatchNorm1d(150),
    nn.SiLU(),
    nn.Linear(150,1,bias=True)
)

In [35]:
optimizer = optim.Adam(model.parameters(),lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)
torch.cuda.empty_cache()
train(model,optimizer,epoch=150)

Train Epoch: 0	Loss: 1759.356689
calculating mse for valid set
valid mse: 57204.37121582031
Train Epoch: 0	Loss: 1482.061035
calculating mse for valid set
valid mse: 54497.81396484375
Train Epoch: 1	Loss: 1407.079102
calculating mse for valid set
valid mse: 53976.50646972656
Train Epoch: 1	Loss: 1454.273560
calculating mse for valid set
valid mse: 44477.78546142578
Train Epoch: 2	Loss: 1407.323120
calculating mse for valid set
valid mse: 43396.65490722656
Train Epoch: 2	Loss: 1577.911621
calculating mse for valid set
valid mse: 32329.328735351562
Train Epoch: 3	Loss: 1060.422241
calculating mse for valid set
valid mse: 31497.922790527344
Train Epoch: 3	Loss: 715.865845
calculating mse for valid set
valid mse: 24534.957458496094
Train Epoch: 4	Loss: 597.019592
calculating mse for valid set
valid mse: 24094.85235595703
Train Epoch: 4	Loss: 1087.064453
calculating mse for valid set
valid mse: 20692.720794677734
Train Epoch: 5	Loss: 605.318726
calculating mse for valid set
valid mse: 20459

In [36]:
model.load_state_dict(torch.load("./Models/FCNN.pt"))
test_mse = check_mse(test_loader,model)
print('Test MSE:', test_mse)
print('Test RMSE:',np.sqrt(test_mse))

calculating mse for valid set
Test MSE: 6793.714263916016
Test RMSE: 82.42399082740423
